# BitBully: Getting Started 🚀

In [ ]:
import bitbully as bb

## Creating Boards

In [ ]:
# Create an empty board
board = bb.Board()  # Empty board
board = bb.Board([[0] * 6 for _ in range(7)])  # also an empty board as 2D list
board

In [ ]:
# Initialize a board with 3 moves in the center column
board = bb.Board([3, 3, 3])
board

In [ ]:
board = bb.Board("33333111")
board

In [ ]:
board.set_board("331")  # this overrides the current board
board.set_board([3, 3, 3])  # this also overrides the current board
board

## Playing Moves

In [ ]:
# Play a sequence of moves into the center column (column index 3)
import bitbully as bb

board = bb.Board()
assert board.play([3, 3, 3])
board

In [ ]:
# Play a sequence of moves across all columns
board = bb.Board()
assert board.play(range(7))
board

In [ ]:
# Play a sequence defined by a string
board = bb.Board()
assert board.play("33333111")
board